In [1]:
api_key='RGAPI-78be8be5-7f0e-4c35-a3bb-53d7951373d7'
import requests

In [2]:
import pandas as pd

root= 'https://euw1.api.riotgames.com/'
challenger_root = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
grandmaster_root = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
master_root = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

challenger_response = requests.get(root+challenger_root+'?api_key='+api_key)
grandmaster_response = requests.get(root+grandmaster_root+'?api_key='+api_key)
master_response = requests.get(root+master_root+'?api_key='+api_key)

challenger_df = pd.DataFrame(challenger_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
grandmaster_df=pd.DataFrame(grandmaster_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
master_df=pd.DataFrame(master_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)


#concatenating the dataframes
high_rank_ladder=pd.concat([challenger_df, grandmaster_df, master_df])[0:100].reset_index(drop=True)
high_rank_ladder=high_rank_ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
high_rank_ladder['rank']+=1

high_rank_ladder

,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,5NzokD2VZC4DB3qBJdl6LOO3PiRQcORDRHZRLgSKdBtt0YA,1681,226,136,True,False,False,True
1,2,jOdw_LVxNtdTh65T_VL7qRzrguVEKKkqQt9CUyL0f7fMdDI,1508,321,237,True,False,False,True
2,3,JGhpg_EeM7jdF-U90nyputqFS-hC3ftrDN---vtbuvo7uDzA,1504,235,159,True,False,False,False
3,4,U4_ndlbviSSK2g20hBy5NNOG1awHFhM2iV_u4NlV73ayAd...,1495,433,341,True,False,False,True
4,5,KFCNaCBCxDEfbGcW3ltdr_of4XStG8Q1CLDY6OOFTcm1Kz0,1469,256,172,True,False,False,False
...,...,...,...,...,...,...,...,...,...
95,96,M0S0vKxHQtjl7UQHGWU0KNkcr6VUFZDMo2I85Ro5bQBt5eMy,1011,187,129,True,False,False,False
96,97,RRt-p_OPdbVep9Vg-48PMa3wz-u37kWWzp_HOI1NPJxRoXE,1011,346,299,True,False,False,False
97,98,xqOBwXph8oj540BUpPlj_fzryuioLNkoxnfSRaMcyYgfluI,1010,162,104,False,False,True,True
98,99,V4AyxtEoGXG5h7mZsgX4aM9BBMOpqOsyw3jVSSGkaLrOhfo,1010,264,211,True,False,False,False


In [3]:
import time
def get_puuid_from_summoner_id(summonerId=None):
    link=f'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}'

    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        puuid=response.json()['puuid']
        return puuid

In [4]:
puuid_list = []

for summoner in high_rank_ladder['summonerId']:
    puuids = get_puuid_from_summoner_id(summonerId=summoner)
    puuid_list.append(puuids)

rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded


In [5]:
puuid_list[0:5]

['buvMukA-JVre-d3tH8WGdH0Fewk5VwFKmuVog33gO7PDRcf_-BV4Iy6jCV0DsaN_HfPU1Or2suJVSA',
 'aZFgbLH6q7UTdHCTLmcBIvarGyRb38eEzW0b_jwnmFx90Ur988DDZvv9kSyoqOedO_rIM5b6Y2VEgg',
 'A4-eBTYpPDIpHuBu2ChYaB7Jq2Ja5YXAW5LBwBjVjljKnmv3kS98yPQXotPL6tURenUmBhUJplH4mA',
 'rH_mM7PpbVDWYWZeAY9_wh_CKd4_lTUbS8dMpMRxfOvZDNp_wQVAG_KGOORmnlDhViUhIyNIuES0hQ',
 'GFaoHL-BXl_WQ7ni6OoBY4PI8vIaPFsLUo5EH0WN5QNX-j63OFSF79byTEhwHsxqsl31Ifpt7AcqwQ']

In [25]:
def get_match_id_from_puuid(puuid=None):
    """
    Function to retrieve the most recent 20 match id's. Takes a player's puuid as argument and outputs the match id's of their most recent 20 ranked matches.
    Queue and type must remain as 420 and ranked respectively as this will only get the ranked matches which is what we want.
    """
    link=f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&type=ranked&start=0&count=20&api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        match_id_list=response.json()
        break
    return match_id_list


In [7]:
all_match_id_list=[]

for player_puuid in puuid_list:
    match_ids=get_match_id_from_puuid(puuid=player_puuid)
    all_match_id_list.extend(match_ids)

In [8]:
all_match_id_list[0:5]

['EUW1_6927403998',
 'EUW1_6926350115',
 'EUW1_6925725972',
 'EUW1_6925715776',
 'EUW1_6925697889']

In [9]:
len(all_match_id_list)

2000

In [10]:
from collections import Counter

# Count the occurrences of each match ID
match_id_counts = Counter(all_match_id_list)

# Find the duplicate match IDs
duplicate_match_ids = [match_id for match_id, count in match_id_counts.items() if count > 1]

# Remove duplicate match IDs from the list
unique_match_id_list = list(set(all_match_id_list))

print(f"Original number of match IDs: {len(all_match_id_list)}")
print(f"Number of duplicate match IDs: {len(duplicate_match_ids)}")
print(f"Number of unique match IDs after removal: {len(unique_match_id_list)}")


Original number of match IDs: 2000
Number of duplicate match IDs: 300
Number of unique match IDs after removal: 1519


In [24]:
unique_match_id_list[0:5]

['EUW1_6924349253',
 'EUW1_6927643216',
 'EUW1_6927762423',
 'EUW1_6909754606',
 'EUW1_6925959671']

In [12]:
def get_match_info_from_id(matchId=None):
    link= f'https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        elif response.status_code==200:
            break
        elif response.status_code==404:
            return 0
        else:
            print('Response is bad: '+str(response.status_code))
    match_data=response.json()
    return match_data

In [13]:
get_match_info_from_id(matchId='EUW1_6927403998')

rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded


{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_6927403998',
  'participants': ['Tov20b07NQJuFhCOsaZjycFjoq18flwe_dON0CKgaMhQmy159u8Rm7upbA25EGYN2GT4K3y5-8jxMA',
   'buvMukA-JVre-d3tH8WGdH0Fewk5VwFKmuVog33gO7PDRcf_-BV4Iy6jCV0DsaN_HfPU1Or2suJVSA',
   'ltQ8rOkidkdVUyU_yw5RwdV-a8vhGNJPQYDz3dHN8yvS7dkUfINBE5YbMDYnuFaE1r6Z-4B8TZ6I9Q',
   'e_ntKn-HGT6_l65X7TlKZDbwvprjpAbgp2hiOG0fPi4Wptjj0I9pvJWswT_m6NgKUcdkJo6B1wwmwQ',
   'uRzn4dbQbN8WqQ0yaMxAT-EZuH8Od2A0f_Byf5viDrp55GLCnqs0AuS6gz1S2OV0l-WGCrEvWHe7KA',
   'PvfgErnSR4iRN3JCTKgkh0GSk5bXkIzkh2xkUpvWg_NIKEmMTd25Qk6xbosFl6uVh8lAwlHWOFJzAA',
   'uYaCDzCl_v0jkf12xwUfOsSfSRMyOYSjOKfipzawzNo_hdKz0TVvwmK4uODKDbO5aX2oeKuchInusA',
   'g0ZhqegHwAgkEkbP6nl2ctb0cDX344SKDFtFXQmB1HoOgGgLHZtZ3F3Qx9W6Cwq3pdMmcxTynQEUug',
   'nbDipOekS8A9bTN0k81h5URA_e2Hx86tHntzK1TZPh891EWIYjZrZ_-AGfmjDczrKpATfNRQv3GCBA',
   'qrdGXPjGFtt9EH-EkyfCCKE-3T-eDKBeKT26KnQPIA5QSn5f1UM6fDK3kMRpj8vDML5K3FJsaRQ_cA']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 171

In [14]:
'''def get_specific_match_info(data):

    
    info = data['info']
    players = info['participants']
    blue_side=data['metadata']['participants'][:5]
    red_side=data['metadata']['participants'][5:]
    game_duration=info['gameDuration']


    total_minions_killed=0
    total_kills=0
    total_deaths=0
    total_assists=0
    total_gold_earned=0
    
    match_data=[]
    # Iterate through each player's data
    for player in players:

        allied_jungle_monster_kills = player['challenges']['alliedJungleMonsterKills']
        enemy_jungle_monster_kills = player['challenges']['enemyJungleMonsterKills']
        minions_killed=player['totalMinionsKilled']
        kills=player['kills']
        assists=player['assists']
        deaths=player['deaths']


        total_minions_killed+=allied_jungle_monster_kills+enemy_jungle_monster_kills+minions_killed
        total_assists+=assists
        total_deaths+=deaths
        total_kills+=kills

    match_data.append({
        'Total minions killed': total_minions_killed,
        'Total deaths': total_deaths,
        'Total assists': total_assists,
        'Total kills': total_kills,
        'Game duration': game_duration
    })

    high_rank_df=pd.DataFrame(match_data)
    return high_rank_df'''

"def get_specific_match_info(data):\n\n    \n    info = data['info']\n    players = info['participants']\n    blue_side=data['metadata']['participants'][:5]\n    red_side=data['metadata']['participants'][5:]\n    game_duration=info['gameDuration']\n\n\n    total_minions_killed=0\n    total_kills=0\n    total_deaths=0\n    total_assists=0\n    total_gold_earned=0\n    \n    match_data=[]\n    # Iterate through each player's data\n    for player in players:\n\n        allied_jungle_monster_kills = player['challenges']['alliedJungleMonsterKills']\n        enemy_jungle_monster_kills = player['challenges']['enemyJungleMonsterKills']\n        minions_killed=player['totalMinionsKilled']\n        kills=player['kills']\n        assists=player['assists']\n        deaths=player['deaths']\n\n\n        total_minions_killed+=allied_jungle_monster_kills+enemy_jungle_monster_kills+minions_killed\n        total_assists+=assists\n        total_deaths+=deaths\n        total_kills+=kills\n\n    match_data

In [15]:
#get_specific_match_info(data=get_match_info_from_id(match_id='EUW1_6898875709'))

In [75]:
import pandas as pd

def get_specific_match_info(data):
    info = data['info']
    metadata = data['metadata']
    participants = metadata['participants']  # List of player puuids
    game_duration = info['gameDuration']

 
    
    #build dictionaries for both blue adn red sides
    blue_side_stats = {
        'minions_killed': 0,
        'kills': 0,
        'deaths': 0,
        'assists': 0,
        'gold_earned': 0,
        'gold_spent': 0,
        'wards_killed':0,
        'wards_placed':0,
        'vision_score':0
    }
    red_side_stats = {
        'minions_killed': 0,
        'kills': 0,
        'deaths': 0,
        'assists': 0,
        'gold_earned': 0,
        'gold_spent': 0,
        'wards_killed':0,
        'wards_placed':0,
        'vision_score':0
    }
    
    #iterate through each players data. if the index of their participant id/puuid is less than 5 then they are on the blue team, otherwise they are on red team
    #side_stats is therefore assigned to each side depending on whether the participant/player id (i) is less than or greater than 5.
    for i, player in enumerate(info['participants']):
        if i < 5:
            side_stats=blue_side_stats
        else:
            side_stats=red_side_stats

        neautral_minions_killed = player['neutralMinionsKilled']
        minions_killed = player['totalMinionsKilled']
        kills = player['kills']
        assists = player['assists']
        deaths = player['deaths']
        gold_earned = player['goldEarned']
        gold_spent = player['goldSpent']
        wards_killed = player['wardsKilled']
        wards_placed=player['wardsPlaced']
        vision_score=player['visionScore']

        #add up the stats for both blue and red sides
        side_stats['minions_killed'] += neautral_minions_killed+minions_killed
        side_stats['assists'] += assists
        side_stats['deaths'] += deaths
        side_stats['kills'] += kills
        side_stats['gold_earned']+=gold_earned
        side_stats['gold_spent']+=gold_spent
        side_stats['wards_killed']+=wards_killed
        side_stats['wards_placed']+=wards_placed
        side_stats['vision_score']+=vision_score

    #create a dataframe with blue and red side stats
    df = pd.DataFrame({
        'game_duration':[game_duration],

        #blueside
        'total_blue_side_minions_killed': [blue_side_stats['minions_killed']],
        'total_blue_side_kills': [blue_side_stats['kills']],
        'total_blue_side_deaths': [blue_side_stats['deaths']],
        'total_blue_side_assists': [blue_side_stats['assists']],
        'total_blue_side_gold_earned':[blue_side_stats['gold_earned']],
        'total_blue_side_gold_spent':[blue_side_stats['gold_spent']],
        'total_blue_side_wards_killed':[blue_side_stats['wards_killed']],
        'total_blue_side_wards_placed':[blue_side_stats['wards_placed']],
        'total_blue_side_vision_score':[blue_side_stats['vision_score']],

        #red side
        'total_red_side_minions_killed': [red_side_stats['minions_killed']],
        'total_red_side_kills': [red_side_stats['kills']],
        'total_red_side_deaths': [red_side_stats['deaths']],
        'total_red_side_assists': [red_side_stats['assists']],
        'total_red_side_gold_earned':[red_side_stats['gold_earned']],
        'total_red_side_gold_spent':[red_side_stats['gold_spent']],
        'total_red_side_ward_takedowns':[red_side_stats['wards_killed']],
        'total_red_side_wards_placed':[red_side_stats['wards_placed']],
        'total_red_side_vision_score':[red_side_stats['vision_score']]
    })

    return df



In [76]:
#test_function(data=get_match_info_from_id(matchId='EUW1_6927403998'))

In [77]:
get_specific_match_info(data=get_match_info_from_id(matchId='EUW1_6927403998'))

,game_duration,total_blue_side_minions_killed,total_blue_side_kills,total_blue_side_deaths,total_blue_side_assists,total_blue_side_gold_earned,total_blue_side_gold_spent,total_blue_side_wards_killed,total_blue_side_wards_placed,total_blue_side_vision_score,total_red_side_minions_killed,total_red_side_kills,total_red_side_deaths,total_red_side_assists,total_red_side_gold_earned,total_red_side_gold_spent,total_red_side_ward_takedowns,total_red_side_wards_placed,total_red_side_vision_score
0,1286,751,26,10,23,49950,45175,18,41,110,615,10,27,15,35267,34300,14,53,105


In [18]:
game=get_match_info_from_id(matchId='EUW1_6917216983')

In [19]:
def test_function(match_data):
    metadata=game['metadata']
    info=game['info']
    players=info['participants']
    match_id=metadata['matchId']
    participants = game['metadata']['participants']
    teams=info['teams']


    game_creation = info['gameCreation']
    game_duration = info['gameDuration']


    for team in teams:
        objectives = team['objectives']
        baron_nash = objectives['baron']
        dragons=objectives['dragon']
        void_grubs = objectives['horde']
        rift_herald = objectives['riftHerald']
        tower = objectives['tower']
        inhibitor=objectives['inhibitor']

        #for objective in [baron_nash, dragons,void_grubs,rift_herald,tower,inhibitor]:
            #first=objective['first']
            #objective_kills=objective['kills']

    match_df = pd.DataFrame({
        'match_id':[match_id],
        'participants':[participants],
        'game creation':[game_creation],
        'game duration':[game_duration]
    })
    return match_df

In [42]:
match_ids = unique_match_id_list[0:5]

df=pd.DataFrame()

for match_id in match_ids:
    game = get_match_info_from_id(matchId=match_id)
    matchdf=get_specific_match_info(game)

    df= pd.concat([df,matchdf])

In [47]:
df.head(3)

,game_duration,total_blue_side_minions_killed,total_blue_side_kills,total_blue_side_deaths,total_blue_side_assists,total_blue_side_gold_earned,total_blue_side_gold_spent,total_red_side_minions_killed,total_red_side_kills,total_red_side_deaths,total_red_side_assists,total_red_side_gold_earned,total_red_side_gold_spent
0,1818,843,37,36,71,65706,58900,913,36,38,49,64166,63025
0,1430,807,33,24,45,57297,53655,735,24,33,30,48336,44735
0,2811,1468,36,62,50,95097,89330,1277,62,36,104,104077,100630
